## Main function test

In [69]:
#-*- coding=utf-8 -*-
import requests
import json
import time
import hashlib
import pandas as pd
from datetime import datetime

URL = "https://s5.securepilot.com/parking/v1/query_record"
api_key = "PARK-eMPpua7y17"
api_secret = "2PHIAGI5qv5IhMGZ56iy"

mac_gemtek = "101a"
mac_address1 = mac_gemtek + "0d0100ab"
mac_address_test = mac_gemtek + "0d01019a"

def byteToMagData(Low_byte, High_byte):
    return twos_comp(byte2SwapToNumber(Low_byte,High_byte), 16)

def byte2SwapToNumber(Low_byte, High_byte):
    return int(Low_byte.encode(), 16) + int(High_byte.encode(), 16) * 256

def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val 

def getParkDataResponse(URL, api_key, api_secret, macID):
    current_unix_time = str(int(time.time()))
    api_token = hashlib.sha1((api_secret + current_unix_time).encode('utf-8')).hexdigest()
    index = 0
    limit = 2
    print("------------- RESTful API parameter --------------")
    print("mac: ", macID)
    print("api_key: ", api_key)
    print("api_secret: ", api_secret)
    print("unix_time: ", current_unix_time)
    print("api_token: ", api_token)
    print("index: ", index, "limit: ", limit)
    print("--------------------------------------------------\n")

    if len(macID) != 12:
        print("mac address is ilegal !!")

#     payload = { "index" : 1, "limit" : 20, "mac" : macID, 
#            "api_key" : api_key, "api_token" : api_token, "time" : current_unix_time
#            }
#     payload = {
#         'mac' : macID, 
#         'api_key' : api_key, 
#         'api_token' : api_token, 
#         'time' : current_unix_time 
#     }
    
    payload = {
        "index": index,
        "limit" : limit,
        "mac" : "101a0d01019a",
        "api_key" : "PARK-eMPpua7y17",
        "api_token" : "7a5db58d81e51ce9cb5a7731d6c6b7fa1bb0144a",
        "time" : "1500874971"
    }
    print("------------- Payload Check ----------------------")
    print("Payload:", payload)
    print("--------------------------------------------------\n")

#     return requests.post(URL, data = payload)
    return requests.post(URL, json = payload)

def getPraseTargetData(req):
    prase_data = prase_data['value']['hits']
    
    prase_data_from_json = []
    # columes in data frame
    prase_data_columes = ['_id', '_index', 'create_at_time','recv_at_time','data',
                          'company','road','slot_num','gwip', 
                          "magX", "magY", "magZ", "magT"]
    # get data by list
    for each in prase_data:
        fullData = []
        pakcetHeader = "fcffff07"
        packetByte = []
        
        fullData = [ each['_id'], each['_index'], 
             each['_source']['create_at'], each['_source']['recv'],
             each['_source']['data'],
             each['_source']['extra']['company'],  each['_source']['extra']['road'], 
             each['_source']['extra']['slot_num'], each['_source']['gwip']
        ]
        
        # get high/ low byte
        packetMagData = each['_source']['data']
        
        if packetMagData.find(pakcetHeader) != -1:
            index = [i for i in range(len(packetMagData)) if i % 2 == 0]   
            for each in index:
                packetByte.append(packetMagData[each:each+2])
#             print("packet Byte: ", packetByte)
        else:
            print("packet header error")

        magX = byteToMagData(packetByte[4], packetByte[5])
        magY = byteToMagData(packetByte[6], packetByte[7])
        magZ = byteToMagData(packetByte[8], packetByte[9])
        magT = byteToMagData(packetByte[10], '00')
        
        fullData.extend([magX, magY, magZ, magT])
        print("Data:", fullData)
        prase_data_from_json.append(fullData)
       
    return pd.DataFrame(prase_data_from_json, columns=prase_data_columes) 


# main funciton
req = getParkDataResponse(URL, api_key, api_secret, mac_address_test)
server_ReturnCode = req.json()['status']['code']
server_ReturnMsg = req.json()['status']['message']

if(req.ok and server_ReturnCode == 1200):
    print("------------- Request Status ---------------------")
    print("Request URL: ",req.url, ", Status: OK ")
    print("--------------------------------------------------\n")
    
    df = getPraseTargetData(req)
    NodeName = df['slot_num'][0]
    timeName = datetime.now().strftime('%Y%m%d_%H%M%S')
    csvFileName = timeName + '_' + NodeName + '.csv'
    df.to_csv(csvFileName, encoding='utf_8_sig')
else:
     print("Status: FAIL , Server return code ",server_ReturnCode, ", Msg :", server_ReturnMsg)
    
# #     print(req.text)



------------- RESTful API parameter --------------
mac:  101a0d01019a
api_key:  PARK-eMPpua7y17
api_secret:  2PHIAGI5qv5IhMGZ56iy
unix_time:  1501139831
api_token:  118dba38cfed8b07d99c576cb04b9ebefb70537c
index:  0 limit:  2
--------------------------------------------------

------------- Payload Check ----------------------
Payload: {'index': 0, 'limit': 2, 'mac': '101a0d01019a', 'api_key': 'PARK-eMPpua7y17', 'api_token': '7a5db58d81e51ce9cb5a7731d6c6b7fa1bb0144a', 'time': '1500874971'}
--------------------------------------------------

------------- Request Status ---------------------
Request URL:  https://s5.securepilot.com/parking/v1/query_record , Status: OK 
--------------------------------------------------

Data: ['AV197R2L-R03vmj0K7en', 'parking_record_index_v2', '2017-07-26T08:04:37.898Z', '2017-07-26T08:04:37.000Z', 'fcffff072d001100f4001c', 'yilan', '宜蘭市中山路二段', '111', '10.5.151.1', 45, 17, 244, 28]
Data: ['AV197Q3r-R03vmj0K7ei', 'parking_record_index_v2', '2017-07-26T08

## API Class

In [36]:
from datetime import datetime
import hashlib

class Gemtek_restful_api:
    def __init__(self, URL, api_key, api_secret, macID, index = 0, limit = 10, start_time = 0):
        self.URL = URL
        self.api_key = api_key
        self.api_secret = api_secret
        self.macID = macID
        self.index = index
        self.limit = limit
        self.time = str(int(time.time()))
        self.start_time  = 0
        self.api_token = hashlib.sha1((self.api_secret + self.time).encode('utf-8')).hexdigest()
        self.payload = {
            "index": self.index,
            "start_time" : self.start_time,
            "time" : self.time,
            "mac" : self.macID,
            "api_key" : self.api_key,
            "api_token" : self.api_token,
            "time" : self.time
        }
        self.jsonData = None
        self.request_OK = None
        if len(macID) != 12:
            print("mac address is ilegal !!")
        
    def set_query_index(self, index = 0):
        self.index = index

    def set_query_limit(self, limit = 10):
        self.limit = limit
        
    def set_query_start_time(self, start_time = 0):
        self.start_time = start_time

    def print_API_parameter(self):
        print("------------- RESTful API parameter --------------")
        print("mac: ", self.macID)
        print("api_key: ", self.api_key)
        print("api_secret: ", self.api_secret)
        print("unix_time: ", self.time)
        print("api_token: ", self.api_token)
        print("index: ", self.index, "limit: ", self.limit, "start_time", self.start_time)
        print("Payload:", self.payload)
        print("--------------------------------------------------\n")
        
    def get_query_data(self):    
        req = requests.post(self.URL, json = self.payload)
        self.jsonData = req.json()
        server_ReturnCode = self.jsonData['status']['code']
        server_ReturnMsg = self.jsonData['status']['message']
        if (server_ReturnCode == 1200):
            self.request_OK = True
        else:
            self.request_OK = False
        
        print("------------- Request Status ---------------------")
        print("Request URL: ", req.url)
        print("Server return code ",server_ReturnCode, ", Msg :", server_ReturnMsg)
        print("--------------------------------------------------\n")
     
        return self.jsonData
    
    def update_new_api_token(self):    
        self.time = str(int(time.time()))
        self.api_token = hashlib.sha1((self.api_secret + self.time).encode('utf-8')).hexdigest()
        self.payload["time"] = self.time
        self.payload["api_token"] = self.api_token
    


In [37]:
# test_api = Gemtek_restful_api(URL, api_key, api_secret, mac_address_test)
# test_api.print_API_parameter()
# query_JsonData = test_api.get_query_data()

In [38]:
from time import mktime
from datetime import datetime

current_time_obj = time.time()
current_datetime_obj = datetime.fromtimestamp(current_time_obj)
current_time = int(current_time_obj)
print(current_time)

1501577268


In [39]:
#-*- coding=utf-8 -*-
import requests
import json
import time
import hashlib
import pandas as pd
from datetime import datetime
from time import mktime
from datetime import datetime

URL = "https://s5.securepilot.com/parking/v1/query_record"
api_key = "PARK-eMPpua7y17"
api_secret = "2PHIAGI5qv5IhMGZ56iy"

mac_gemtek = "101a"
mac_address1 = "0d0100ab"
mac_address_test = "0d01019a"

mac_query = mac_gemtek + mac_address_test

def byteToMagData(Low_byte, High_byte):
    value = int(Low_byte.encode(), 16) + int(High_byte.encode(), 16) * 256
    return twos_comp(value, 16)

def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val 

def getPraseTargetData(Data):
    prase_data = Data['value']['hits']
    prase_data_from_json = []
    
    # columes in data frame
    prase_data_columes = ['_id', '_index', 'mac','create_at_time','recv_at_time','frameCnt', 'data',
                          'company','road','slot_num','gwip', 
                          "magX", "magY", "magZ", "magT"]
    # get data by list
    for each in prase_data:
        fullData = []
        pakcetHeader = "fcffff07"
        packetByte = []
        
        fullData = [ each['_id'], each['_index'], each['_source']['macAddr'],
             each['_source']['create_at'], each['_source']['recv'], each['_source']['frameCnt'],
             each['_source']['data'], 
             each['_source']['extra']['company'],  each['_source']['extra']['road'], 
             each['_source']['extra']['slot_num'], each['_source']['gwip']
        ]
        
        # get high/ low byte
        packetMagData = each['_source']['data']
        
        if packetMagData.find(pakcetHeader) != -1:
            index = [i for i in range(len(packetMagData)) if i % 2 == 0]   
            for each in index:
                packetByte.append(packetMagData[each:each+2])
#             print("packet Byte: ", packetByte)
        else:
            print("packet header error")

        magX = byteToMagData(packetByte[4], packetByte[5])
        magY = byteToMagData(packetByte[6], packetByte[7])
        magZ = byteToMagData(packetByte[8], packetByte[9])
        magT = byteToMagData(packetByte[10], '00')
        
        fullData.extend([magX, magY, magZ, magT])
        print("Data:", fullData)
        prase_data_from_json.append(fullData)
       
    return pd.DataFrame(prase_data_from_json, columns=prase_data_columes) 

# main funciton
test_api = Gemtek_restful_api(URL, api_key, api_secret, mac_query, index = 0 , limit = 200, start_time = current_time)
test_api.print_API_parameter()
query_JsonData = test_api.get_query_data()

if(test_api.request_OK):
    df = getPraseTargetData(query_JsonData)
    NodeName = mac_query
    timeName = current_datetime_obj.strftime('%Y%m%d_%H%M%S')
    csvFileName = 'ID_' + NodeName + '_' + timeName + '.csv'
    df.to_csv(csvFileName, encoding='utf_8_sig')
else:
    print("Status: FAIL , Server return code ",server_ReturnCode, ", Msg :", server_ReturnMsg)
    
# #     print(req.text)



------------- RESTful API parameter --------------
mac:  101a0d01019a
api_key:  PARK-eMPpua7y17
api_secret:  2PHIAGI5qv5IhMGZ56iy
unix_time:  1501577269
api_token:  a3ae65cf468d1d484df896c33ad8a1d1cdcf2baf
index:  0 limit:  200 start_time 0
Payload: {'index': 0, 'start_time': 0, 'time': '1501577269', 'mac': '101a0d01019a', 'api_key': 'PARK-eMPpua7y17', 'api_token': 'a3ae65cf468d1d484df896c33ad8a1d1cdcf2baf'}
--------------------------------------------------

------------- Request Status ---------------------
Request URL:  https://s5.securepilot.com/parking/v1/query_record
Server return code  1200 , Msg : Command succeeded
--------------------------------------------------

Data: ['AV2czgGy-R03vmj0MBst', 'parking_record_index_v2', '101a0d01019a', '2017-08-01T07:58:52.849Z', '2017-08-01T07:58:52.000Z', 24, 'fcffff0788ff34ff78ff1e', 'yilan', '宜蘭市中山路二段', '111', '10.5.151.1', -120, -204, -136, 30]
Data: ['AV2czfYc-R03vmj0MBsr', 'parking_record_index_v2', '101a0d01019a', '2017-08-01T07:58:4

In [27]:
datetime.now()

datetime.datetime(2017, 8, 1, 16, 43, 25, 589918)

In [28]:
time.time()

1501577039.181712

In [7]:
query_JsonData

{'status': {'code': 1200, 'message': 'Command succeeded'},
 'value': {'hits': [{'_id': 'AV2cuzmt-R03vmj0MBcL',
    '_index': 'parking_record_index_v2',
    '_score': None,
    '_source': {'channel': 922400000,
     'commsysType': 'lora',
     'create_at': '2017-08-01T07:38:21.996Z',
     'data': 'fcffff0786003000adff1d',
     'doc_type': 'parking_record',
     'extra': {'company': 'yilan',
      'road': '宜蘭市中山路二段',
      'slot_num': '111',
      'type': 'node'},
     'fport': 1,
     'frameCnt': 205,
     'gwid': '00001c497b431e2e',
     'gwip': '10.5.151.1',
     'macAddr': '101a0d01019a',
     'recv': '2017-08-01T07:38:21.000Z',
     'repeater': '00000000ffffffff',
     'rssi': -52,
     'sf': 10,
     'snr': 27,
     'snr_max': 35,
     'snr_min': 20,
     'systype': 13},
    '_type': 'parking_record_type'},
   {'_id': 'AV2cuzlp-R03vmj0MBcK',
    '_index': 'parking_record_index_v2',
    '_score': None,
    '_source': {'channel': 926375000,
     'commsysType': 'lora',
     'create_at

In [ ]:
import matplotlib.pyplot as plt


df.to_csv(csvFileName, encoding='utf_8_sig')

plt.plot([1,2,3,4])
plt.ylabel('some numbers')
plt.show()